In [1]:
from copy import deepcopy

import pandas as pd
import networkx as nx

import pm4py

In [3]:
dcr_source = {
    'events': set(['A','B','C','D','E','F']),
    'conditionsFor': {'A':{'B'},'C':{'B'}},
    'milestonesFor': {},
    'responseTo': {},
    'noResponseTo': {},
    'includesTo': {'A':{'D','E','F'},'B':{'D','E','F'}},
    'excludesTo': {'C':{'D','E','F'}},
    'marking': {'executed': set(),
                'included': set(['A','B','C','D','E','F']),
                'pending': set(),
                'executedTime': {},  # Gives the time since a event was executed
                'pendingDeadline': {}  # The deadline until an event must be executed
                },
    'conditionsForDelays': {},
    'responseToDeadlines': {},
    'subprocesses': {},
    'nestings': {},
    'labels': set(),
    'labelMapping': {},
    'roles': set(),
    'roleAssignments': {},
    'readRoleAssignments': {}
}

dcr = deepcopy(dcr_source)

In [ ]:
# def build_rel_matrices_old(dcr):
#     rel_matrices = {}
#     ind = pd.Index(sorted(dcr['events']),dtype=str)
#     edge_weights = pd.DataFrame(0,columns=ind,index=ind,dtype=int)
#     for rel in ['conditionsFor', 'responseTo', 'includesTo', 'excludesTo', 'milestonesFor','noResponseTo']:
#         rel_matrix = pd.DataFrame(0,columns=ind,index=ind,dtype=int)
#         for e in dcr['events']:
#             for e_prime in dcr['events']:
#                 if e in dcr[rel] and e_prime in dcr[rel][e]:
#                     rel_matrix.at[e,e_prime] = 1
#         edge_weights = edge_weights.add(rel_matrix,fill_value=0)
#         rel_matrices[rel] = rel_matrix
#     rel_matrices['all'] = edge_weights
#     return rel_matrices
# 
# def common_out_neighbors(g, i, j):
#     return set(g.successors(i)).intersection(g.successors(j))
# 
# def common_in_neighbors(g, i, j):
#     return set(g.predecessors(i)).intersection(g.predecessors(j))
# # for all relations get pairs of nodes that share at least a common successor or predecessor in this adjencency graph
# def find_nestings_old(dcr, rel_matrices):
#     nestings_per_rel = {}
#     relations_to_remove = {}
#     new_events = set()
#     relations_to_add = {}
#     i = 0
#     for rel in ['conditionsFor','responseTo','excludesTo','includesTo','noResponseTo','milestonesFor']:
#         nestings = {}
#         # G = None
#         new_nestings_found = True
#         # if not G:
#         G = nx.from_pandas_adjacency(rel_matrices[rel],create_using=nx.DiGraph)
#         relations_to_remove[rel] = {}
#         relations_to_add[rel] = {}
#         while new_nestings_found:
#             new_arrows = set()
#             new_nodes = set()
#             arrows_to_remove = set()
#             largest_nesting = {}
#             # sem_for_nestings = semantics_obj.DcrSemantics(dcr)
#             for e in dcr['events']:
#                 largest_nesting[e] = 0
#             # they have to be top level events
#             for e in G.nodes:
#                 for e_prime in G.nodes:
#                     if e!=e_prime:
#                         cin = common_in_neighbors(G,e,e_prime)
#                         cin.discard(e)
#                         cin.discard(e_prime)
#                         # cin = cin.intersection(sem_for_nestings.tl_events)
#                         con = common_out_neighbors(G,e,e_prime)
#                         con.discard(e)
#                         con.discard(e_prime)
#                         # con = con.intersection(sem_for_nestings.tl_events)
#                         if len(cin) > 0 or len(con) > 0:
#                             if len(cin)>0:
#                                 for c in cin:
#                                     largest_nesting[c] += 1
#                             if len(con)>0:
#                                 for c in con:
#                                     largest_nesting[c] += 1
#             if max(largest_nesting.values())>0:
#                 e = max(largest_nesting,key=largest_nesting.get)
#                 suc_e = set(G.successors(e))
#                 if len(suc_e)>1:
#                     nestings[f'N{i}'] = suc_e
#                     # dcr['events'].add(f'N{i}')
#                     # dcr['subprocesses'][f'N{i}'] = suc_e
#                     new_events.add(f'N{i}')
#                     new_nodes.add(f'N{i}')
#                     new_arrows.add((e, f'N{i}'))
#                     i += 1
#                     for s in suc_e:
#                         arrows_to_remove.add((e, s))
#                 pred_e = set(G.predecessors(e))
#                 if len(pred_e)>1:
#                     nestings[f'N{i}'] = pred_e
#                     # dcr['events'].add(f'N{i}')
#                     # dcr['subprocesses'][f'N{i}'] = pred_e
#                     new_events.add(f'N{i}')
#                     new_nodes.add(f'N{i}')
#                     new_arrows.add((f'N{i}',e))
#                     for p in pred_e:
#                         arrows_to_remove.add((p, e))
#                     i += 1
#     
#                 relations_to_remove[rel].update(arrows_to_remove)
#                 relations_to_add[rel].update(new_arrows)
#                 for k,v in arrows_to_remove:
#                     G.remove_edge(k,v)
#                 for k in new_nodes:
#                     G.add_node(k)
#                 for k,v in new_arrows:
#                     G.add_edge(k,v)
#             else:
#                 new_nestings_found = False
#     
#             new_nestings_found = False
#     
#         nestings_per_rel[rel] = nestings
#     return nestings_per_rel, new_events, relations_to_remove, relations_to_add
# 
# def get_valid_nestings(proposed_nestings):
#     valid_nestings = {}
#     equal_nestings = {}
#     
#     for rel in ['conditionsFor','responseTo','excludesTo','includesTo','noResponseTo','milestonesFor']:
#         for k, v in proposed_nestings[rel].items():
#             valid_nestings[k] = set()
#     
#     for rel in ['conditionsFor','responseTo','excludesTo','includesTo','noResponseTo','milestonesFor']:
#         for rel1 in ['conditionsFor','responseTo','excludesTo','includesTo','noResponseTo','milestonesFor']:
#             for k, v in proposed_nestings[rel].items():
#                 for k1, v1 in proposed_nestings[rel1].items():
#                     if k!=k1 and rel!=rel1:
#                         if 0 < len(v.intersection(v1)) < len(v):
#                             print('not disjoint not the same')
#                             valid_nestings[k] = v.intersection(v1)
#                             valid_nestings[k1] = v.intersection(v1)
#                         else:
#                             add_new_set = True
#                             for j,z in valid_nestings.items():
#                                 if j!=k and z.intersection(v) == z and len(z)>0:
#                                     add_new_set = False
#                                     equal_nestings[j]=k
#                                     break
#                             if add_new_set:
#                                 valid_nestings[k] = v
#     retVal = {}
#     for k,v in valid_nestings.items():
#         if len(v)!=0:
#             retVal[k] = v
#     return retVal, equal_nestings
# dcr = deepcopy(dcr_source)
# rel_matrices = build_rel_matrices(dcr)
# nestings_per_rel, new_events, relations_to_remove, relations_to_add = find_nestings(dcr, rel_matrices)
# print(nestings_per_rel)
# valid_nestings, equal_nestings = get_valid_nestings(nestings_per_rel)
# print(valid_nestings)
# print(equal_nestings)

# For CAISE 2023 - Nesting, patterns, choices, simplicity

In [3]:
from itertools import product
all_rels = ['conditionsFor', 'responseTo', 'includesTo', 'excludesTo', 'milestonesFor','noResponseTo']

def build_rel_matrix(dcr):
    mat_keys = product(dcr['events'],set(all_rels))
    ind = pd.MultiIndex.from_tuples(sorted(mat_keys))
    rel_matrix = pd.DataFrame(0,columns=ind,index=ind,dtype=int)
    for rel in ['conditionsFor', 'responseTo', 'includesTo', 'excludesTo', 'milestonesFor','noResponseTo']:
        for e in dcr['events']:
            for e_prime in dcr['events']:
                if e in dcr[rel] and e_prime in dcr[rel][e]:
                    rel_matrix.at[(e,rel),(e_prime,rel)] = 1

    return rel_matrix
def common_out_neighbors(g, i, j):
    return set(g.successors(i)).intersection(g.successors(j))

def common_in_neighbors(g, i, j):
    return set(g.predecessors(i)).intersection(g.predecessors(j))
# for all relations get pairs of nodes that share at least a common successor or predecessor in this adjencency graph
def find_nestings(events, rel_matrix):
    nestings = {}
    nestings_events = {}
    relations_to_remove = {}
    new_events = set()
    relations_to_add = {}
    
    i = 0
    new_nestings_found = True
    G = nx.from_pandas_adjacency(rel_matrix,create_using=nx.DiGraph)
    
    while new_nestings_found:
        new_arrows = set()
        new_nodes = set()
        arrows_to_remove = set()
        largest_nesting = {}
        for e in events:
            for rel in all_rels:
                largest_nesting[(e,rel)] = 0
        for e in G.nodes:
            for e_prime in G.nodes:
                if e!=e_prime:
                    cin = common_in_neighbors(G,e,e_prime)
                    cin.discard(e)
                    cin.discard(e_prime)
                    con = common_out_neighbors(G,e,e_prime)
                    con.discard(e)
                    con.discard(e_prime)
                    if len(cin) > 0 or len(con) > 0:
                        if len(cin)>0:
                            for c in cin:
                                largest_nesting[c] += 1
                        if len(con)>0:
                            for c in con:
                                largest_nesting[c] += 1
        if max(largest_nesting.values())>0:
            e = max(largest_nesting, key=largest_nesting.get)
            suc_e = set(G.successors(e))
            suc_ee = set([e[0] for e in suc_e])
            pred_e = set(G.predecessors(e))
            pred_ee = set([e[0] for e in pred_e])
            if len(suc_e)>1:
                add = True
                for nk, nesting in nestings_events.items():
                    if nesting==suc_ee:
                        add = False
                if add:
                    nestings[f'N{i}'] = suc_e
                    nestings_events[f'N{i}'] = suc_ee
                    new_events.add(f'N{i}')
                    new_nodes.add(f'N{i}')
                    new_arrows.add((e, f'N{i}'))
                for s in suc_e:
                    arrows_to_remove.add((e, s))
                i += 1
            if len(pred_e)>1:
                add = True
                for nk, nesting in nestings_events.items():
                    if nesting==pred_ee:
                        add = False
                if add:
                    nestings[f'N{i}'] = pred_e
                    nestings_events[f'N{i}'] = pred_ee
                    new_events.add(f'N{i}')
                    new_nodes.add(f'N{i}')
                    new_arrows.add((f'N{i}',e))
                for p in pred_e:
                    arrows_to_remove.add((p, e))
                i += 1

            relations_to_remove.update(arrows_to_remove)
            relations_to_add.update(new_arrows)
            for k,v in arrows_to_remove:
                G.remove_edge(k,v)
            for k in new_nodes:
                G.add_node(k)
            for k,v in new_arrows:
                G.add_edge(k,v)
        else:
            new_nestings_found = False
    
    return nestings_events, new_events, relations_to_remove, relations_to_add

In [4]:
find_nestings(dcr['events'],build_rel_matrix(dcr))

NameError: name 'dcr' is not defined

In [101]:
from itertools import product
all_rels = ['conditionsFor', 'responseTo', 'includesTo', 'excludesTo', 'milestonesFor','noResponseTo']

def build_rel_matrix(dcr):
    mat_keys = product(dcr['events'],set(all_rels))
    ind = pd.MultiIndex.from_tuples(sorted(mat_keys))
    rel_matrix = pd.DataFrame(0,columns=ind,index=ind,dtype=int)
    for rel in all_rels:
        for e in dcr['events']:
            for e_prime in dcr['events']:
                if e in dcr[rel] and e_prime in dcr[rel][e]:
                    rel_matrix.at[(e,rel),(e_prime,rel)] = 1

    return rel_matrix
def common_out_neighbors(g, i, j):
    return set(g.successors(i)).intersection(g.successors(j))

def common_in_neighbors(g, i, j):
    return set(g.predecessors(i)).intersection(g.predecessors(j))
# for all relations get pairs of nodes that share at least a common successor or predecessor in this adjencency graph
def find_nestings(events, rel_matrix):
    nestings = {}
    nestings_events = {}
    relations_to_remove = {}
    new_events = set()
    relations_to_add = {}
    
    i = 0
    new_nestings_found = True
    G = nx.from_pandas_adjacency(rel_matrix,create_using=nx.DiGraph)
    
    while new_nestings_found:
        new_arrows = set()
        new_nodes = set()
        arrows_to_remove = set()
        largest_nesting = {}
        for e in events:
            for rel in all_rels:
                largest_nesting[(e,rel)] = 0
        for e in G.nodes:
            for e_prime in G.nodes:
                if e!=e_prime:
                    cin = common_in_neighbors(G,e,e_prime)
                    cin.discard(e)
                    cin.discard(e_prime)
                    con = common_out_neighbors(G,e,e_prime)
                    con.discard(e)
                    con.discard(e_prime)
                    if len(cin) > 0 or len(con) > 0:
                        if len(cin)>0:
                            for c in cin:
                                largest_nesting[c] += 1
                        if len(con)>0:
                            for c in con:
                                largest_nesting[c] += 1
        if max(largest_nesting.values())>0:
            e = max(largest_nesting, key=largest_nesting.get)
            suc_e = set(G.successors(e))
            suc_ee = set([e[0] for e in suc_e])
            pred_e = set(G.predecessors(e))
            pred_ee = set([e[0] for e in pred_e])
            if len(suc_e)>1:
                add = True
                for nk, nesting in nestings_events.items():
                    if nesting==suc_ee:
                        add = False
                if add:
                    nestings[f'N{i}'] = suc_e
                    nestings_events[f'N{i}'] = suc_ee
                    new_events.add(f'N{i}')
                    new_nodes.add(f'N{i}')
                    new_arrows.add((e, f'N{i}'))
                for s in suc_e:
                    arrows_to_remove.add((e, s))
                i += 1
            if len(pred_e)>1:
                add = True
                for nk, nesting in nestings_events.items():
                    if nesting==pred_ee:
                        add = False
                if add:
                    nestings[f'N{i}'] = pred_e
                    nestings_events[f'N{i}'] = pred_ee
                    new_events.add(f'N{i}')
                    new_nodes.add(f'N{i}')
                    new_arrows.add((f'N{i}',e))
                for p in pred_e:
                    arrows_to_remove.add((p, e))
                i += 1

            relations_to_remove.update(arrows_to_remove)
            relations_to_add.update(new_arrows)
            for k,v in arrows_to_remove:
                G.remove_edge(k,v)
            for k in new_nodes:
                G.add_node(k)
            for k,v in new_arrows:
                G.add_edge(k,v)
        else:
            new_nestings_found = False
    
    return nestings_events, new_events, relations_to_remove, relations_to_add

{'events': {'A', 'B', 'C', 'D', 'E', 'F'},
 'conditionsFor': {'A': {'B'}, 'C': {'B'}},
 'milestonesFor': {},
 'responseTo': {},
 'noResponseTo': {},
 'includesTo': {'A': {'D', 'E', 'F'}, 'B': {'D', 'E', 'F'}},
 'excludesTo': {'C': {'D', 'E', 'F'}},
 'marking': {'executed': set(),
  'included': {'A', 'B', 'C', 'D', 'E', 'F'},
  'pending': set(),
  'executedTime': {},
  'pendingDeadline': {}},
 'conditionsForDelays': {},
 'responseToDeadlines': {},
 'subprocesses': {},
 'nestings': {},
 'labels': set(),
 'labelMapping': {},
 'roles': set(),
 'roleAssignments': {},
 'readRoleAssignments': {}}

In [7]:
from pm4py.algo.discovery.dcr_discover import algorithm
log = pm4py.read_xes('/home/vco/Projects/pm4py-dcr/datasets/rtfmp.xes',return_legacy_log_object=True)
rtfmp_reference_dcr, _ = algorithm.apply(log)
rtfmp_rel_matrix = build_rel_matrix(rtfmp_reference_dcr)
nestings_rtfmp, new_events, relations_to_remove, relations_to_add = find_nestings(rtfmp_reference_dcr, rtfmp_rel_matrix)
print(nestings_rtfmp)

NameError: name 'build_rel_matrices' is not defined